In [1]:
# stolen from here: https://towardsdatascience.com/text-classification-predicting-good-or-bad-statements-using-natural-language-processing-e3a4edb07118

In [1]:
import pandas as pd
df = pd.read_csv("https://gist.githubusercontent.com/johnidm/582cfeadd2bf418df4539c9422f824d2/raw/twitter-sentiment-pt-BR-md-2-l.csv")
df.head()

,tweet_text,sentiment
0,Mas tu não és feio :( @SavageFluxXx__,0
1,@SamaraPaivas Você que pensa :),1
2,te amo demais :( https://t.co/leUzS65WrG,0
3,@nicko_donis lindo! :),1
4,"@B_kirikihira Oi, tem sim! Visite nossos canai...",1


In [2]:
import string
import re
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
nlp = spacy.blank("pt")
REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"
REGX_URL = r"https?://[A-Za-z0-9./]+"
def preprocessing(text):
  text = text.lower()
  
  text = re.sub(REGX_USERNAME, ' ', text)
  text = re.sub(REGX_URL, ' ', text)
  
  emojis = {
      ':)': 'emocaopositiva',
      ':(': 'emocaonegativa'
  }
  
  for e in emojis:
    text = text.replace(e, emojis[e])
  
  tokens = [token.text for token in nlp(text)]
  
  tokens = [t for t in tokens if 
              t not in STOP_WORDS and 
              t not in string.punctuation and 
              len(t) > 3]
  
  tokens = [t for t in tokens if not t.isdigit()]
    
  return " ".join(tokens)
df["tweet_text_clean"] = df["tweet_text"].apply(preprocessing)
df.head()

,tweet_text,sentiment,tweet_text_clean
0,Mas tu não és feio :( @SavageFluxXx__,0,feio emocaonegativa
1,@SamaraPaivas Você que pensa :),1,pensa emocaopositiva
2,te amo demais :( https://t.co/leUzS65WrG,0,emocaonegativa
3,@nicko_donis lindo! :),1,lindo emocaopositiva
4,"@B_kirikihira Oi, tem sim! Visite nossos canai...",1,visite canais saiba projeto incrivel emocaopos...


In [3]:
dataset = list(df[["tweet_text_clean", "sentiment"]].sample(frac=1).itertuples(index=False, name=None))
train_data = dataset[:15000]
dev_data = dataset[15000:18000]
test_data = dataset[18000:]

train_data

[('saudades zayn direction emocaonegativa', 0),
 ('emocaopositiva 1f5856ea-e19f-43cc-a842-8e6a331fcd91', 1),
 ('....', 1),
 ('ótimo certo', 1),
 ('     pfvor emocaonegativa', 0),
 ('no0o0o0o0o emocaonegativa', 0),
 ('menina emocaopositiva', 1),
 ('     emocaopositiva', 1),
 ('foda jogado emocaopositiva foda', 1),
 ('acabei abrir intervalo quase gente sala emocaopositiva', 1),
 ('crush pessoa descobres insta queres seguir queres ache louca emocaopositiva',
  1),
 ('fiquei feliz lembrei-me viver portugal emocaopositiva', 1),
 ('devia tendo acordar emocaopositiva', 1),
 ('acordei trabalhar deveria mundo mundo arrasado falecimento emocaonegativa conseguir trabalhar direito sabendo casa mundo foda',
  0),
 ('grta fiquei tesão emocaopositiva', 1),
 ('fundo poço emocaonegativa abracemos', 0),
 ('linda podemos date thankss quiseres ahaha preciso emocaopositiva', 1),
 ('emocaonegativa', 0),
 ('nóis', 1),
 ("snow efeitos icônicos vídeos emocaopositiva filmorago formato móvel easy edição vídeo ma

In [5]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        doc.cats["NEG"] = label == 0
        print(doc.cats["NEG"])
        db.add(doc)
    
    db.to_disk(outfile)
convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

True
False
False
False
True
True
False
False
False
False
False
False
False
True
False
True
False
True
False
False
False
True
False
True
True
True
False
False
False
False
False
False
False
True
False
False
False
True
True
True
True
False
False
False
False
False
False
True
False
True
True
True
True
True
True
True
True
False
False
False
False
False
True
False
False
True
False
True
False
True
True
False
True
True
False
True
False
True
False
False
False
True
True
True
False
False
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
False
True
True
False
False
False
True
False
True
True
False
False
True
True
False
False
False
False
True
True
True
False
False
False
True
False
False
True
False
True
False
False
False
False
False
False
True
True
True
True
False
True
False
True
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
True
False
False
True
False
True
False
True
False
False
False
False
False
False
False
True
False
F

False
True
True
True
False
False
False
False
True
True
False
False
True
True
False
False
False
False
False
False
True
True
True
True
False
False
True
False
True
False
True
True
False
True
True
True
False
False
True
False
False
True
False
True
False
False
False
True
False
False
False
False
False
False
False
True
False
True
False
False
False
True
True
True
True
False
False
True
False
True
True
True
False
False
True
True
True
True
True
False
True
True
True
False
False
True
True
False
True
False
False
True
True
True
True
False
False
False
True
True
False
False
False
False
True
True
False
True
True
False
False
False
True
False
False
True
False
False
False
False
True
False
True
False
False
False
False
False
False
True
False
False
True
False
True
False
False
False
True
False
False
True
False
True
False
True
False
False
True
False
True
False
False
True
False
False
False
True
False
False
False
True
True
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
Fals

False
True
False
True
False
False
False
True
False
False
True
True
False
True
False
True
False
True
True
False
False
True
True
True
False
False
False
True
True
False
True
False
False
False
True
True
True
False
True
True
True
False
True
True
False
True
False
True
False
True
False
False
True
True
False
False
True
False
True
False
False
True
True
False
True
True
True
True
True
True
False
False
False
True
False
False
False
True
True
True
True
True
False
False
True
False
False
True
False
False
False
True
True
False
False
True
True
False
True
False
True
True
False
True
True
True
True
False
True
True
True
False
False
True
True
False
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
False
True
True
True
False
False
True
True
True
False
False
False
False
True
False
True
False
True
True
True
False
False
True
True
True
True
False
False
False
True
False
True
True
True
True
False
False
True
True
True
True
True
False
True
True
True
Fals

True
True
False
False
False
True
True
True
False
True
True
True
True
False
True
True
True
False
False
False
False
True
False
True
False
True
True
True
True
False
False
True
False
True
True
True
False
True
True
True
False
False
True
False
False
True
False
True
False
True
False
True
True
True
True
False
True
False
True
True
False
True
True
False
False
False
False
True
True
True
False
False
True
True
False
False
True
True
True
True
True
True
True
False
True
False
False
True
False
True
True
False
True
False
True
False
True
False
True
False
True
True
False
True
True
True
True
False
False
True
True
False
True
False
True
True
False
True
True
True
False
False
False
False
False
False
False
False
False
False
False
True
True
False
True
True
True
False
True
True
False
True
False
False
False
True
True
True
True
True
False
False
False
False
False
False
True
False
False
True
True
False
True
False
True
False
False
True
True
False
True
False
True
False
False
False
False
True
False
False
True
True
True


False
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
False
True
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
True
False
False
False
False
True
True
True
False
True
False
True
True
False
False
False
True
True
False
True
True
True
True
False
True
True
True
True
True
False
False
False
True
True
True
False
False
False
True
True
False
True
False
False
False
False
True
True
False
True
True
False
True
True
False
True
True
True
False
True
True
True
False
True
True
True
True
True
True
False
False
False
True
True
False
True
True
False
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
True
False
True
True
True
True
True
False
False
True
True
False
False
True
False
True
True
True
True
False
True
False
False
False
True
False
True
False
False
False
False
False
True
False
True
False
False
False
True
False
True
True
True
True
True
Tru

...

True
False
True
True
False
False
False
False
False
False
True
True
False
False
False
False
False
False
True
False
False
False
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
False
True
False
False
False
False
False
True
False
True
True
False
False
True
False
True
False
True
False
False
True
False
False
True
False
False
False
False
True
True
False
True
False
True
False
False
True
True
True
False
True
True
True
False
False
False
True
True
False
True
True
True
True
False
False
True
True
False
False
True
False
True
True
True
True
True
True
False
True
False
False
False
False
True
False
False
True
True
False
True
False
True
True
False
True
False
False
True
False
False
False
False
False
False
True
False
False
False
False
False
True
True
True
True
True
False
False
True
False
True
False
False
True
False
False
False
True
True
False
True
True
True
True
True
True
False
False
False
False
False
True
True
True
True
False
True
True
False
False
False
False
True
True
True
Fa

False
True
False
True
True
False
False
False
False
False
False
True
True
False
False
True
False
False
False
True
True
True
False
True
False
False
True
True
True
False
True
True
True
True
False
True
True
True
False
True
False
True
False
True
True
True
False
True
True
True
True
False
False
True
False
False
True
False
True
False
True
False
True
False
False
True
True
True
True
False
False
True
True
True
False
True
False
False
True
False
True
False
True
False
True
False
False
False
False
True
True
False
True
True
True
False
True
True
False
True
True
False
False
False
True
True
True
True
True
False
True
True
True
False
True
False
True
False
True
False
False
False
True
True
True
True
True
True
False
False
False
True
False
False
False
False
True
False
True
False
False
True
False
False
True
False
True
False
False
False
True
True
True
True
True
False
False
True
False
False
False
True
False
False
False
True
False
True
True
False
False
True
True
False
False
True
True
True
False
False
False
True
Tr

True
False
False
True
False
False
True
True
False
False
False
False
False
False
True
False
True
True
True
False
False
False
True
True
True
True
False
False
True
True
False
True
True
True
False
True
True
False
False
False
False
False
False
False
True
True
True
False
True
False
True
True
False
False
False
False
True
True
True
True
True
False
False
True
True
False
False
True
True
False
False
True
True
True
False
False
True
False
True
False
True
False
True
False
False
True
True
False
False
False
True
True
False
False
True
True
True
False
True
False
False
True
False
True
True
True
False
False
True
True
False
False
True
False
False
True
False
True
False
False
False
False
False
False
True
False
False
True
True
True
True
False
False
False
False
True
False
True
False
False
True
True
False
False
False
True
True
True
False
False
False
False
True
False
True
True
False
False
True
False
True
True
True
False
False
False
False
True
False
False
True
False
False
True
True
True
True
True
False
True
False

True
False
True
False
True
True
True
False
False
False
False
True
True
False
False
True
False
False
True
True
False
True
True
True
True
False
True
False
False
False
False
False
False
False
True
True
False
True
False
True
True
True
True
False
False
False
False
True
False
False
True
False
True
True
True
True
True
True
True
True
False
True
False
False
False
True
True
True
True
False
False
False
True
False
True
True
True
False
False
False
False
True
True
False
False
True
False
True
False
False
True
True
True
True
False
True
False
True
False
False
True
False
False
True
False
False
False
False
True
False
True
True
True
False
True
True
True
True
False
False
True
True
True
True
True
False
False
True
False
True
True
False
False
True
False
False
True
False
True
True
False
True
False
False
True
True
False
False
False
False
False
False
True
False
False
False
True
True
False
False
False
False
False
False
False
True
True
False
True
False
True
False
True
False
False
False
True
True
False
False
False


True
True
False
True
True
True
True
False
False
True
True
True
False
False
False
True
True
False
False
True
False
False
True
False
False
True
False
True
False
True
False
True
False
False
False
False
False
True
False
False
False
False
False
True
True
True
True
True
False
False
False
True
False
False
False
True
False
True
False
False
False
False
True
False
False
True
False
False
True
True
False
False
False
True
True
True
False
False
True
False
True
True
False
True
False
True
True
True
False
False
True
True
False
False
True
False
False
True
False
True
False
True
True
False
True
True
False
False
False
True
True
True
False
True
True
True
False
False
False
True
True
True
False
False
False
False
False
False
False
False
True
False
True
False
False
True
True
True
False
True
True
True
False
False
False
True
True
False
True
False
True
False
False
False
True
False
False
False
False
False
True
True
True
True
True
False
True
True
True
True
True
False
False
True
True
True
False
True
True
False
False


In [6]:
!python -m spacy init config  --lang pt --pipeline textcat --optimize efficiency --force config.cfg

Traceback (most recent call last):
  File "C:\Users\Emily\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Emily\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Emily\anaconda3\lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "C:\Users\Emily\anaconda3\lib\site-packages\spacy\cli\_util.py", line 74, in setup_cli
    command(prog_name=COMMAND)
  File "C:\Users\Emily\anaconda3\lib\site-packages\click\core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "C:\Users\Emily\anaconda3\lib\site-packages\typer\core.py", line 778, in main
    return _main(
  File "C:\Users\Emily\anaconda3\lib\site-packages\typer\core.py", line 190, in _main
    args = click.utils._expand_args(args)
AttributeError: module 'click.utils' has no attribute '_expand_args'


In [15]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model --verbose

[+] Created output directory: model
[i] Saving to output directory: model
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['textcat']
[i] Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       85.29    0.85
  0     200         33.79       97.29    0.97
  0     400         14.76       98.32    0.98
  0     600          7.42       98.96    0.99
  1     800          4.11       99.06    0.99
  1    1000          2.93       98.99    0.99
  2    1200          2.11       98.89    0.99
  3    1400          1.72       98.93    0.99
  4    1600          1.42       98.93    0.99
  5    1800          1.22       98.96    0.99
  7    2000          1.03       98.93    0.99
  9    2200          0.92       98.96    0.99
 11    2400          0.76    

[2023-04-24 19:32:33,882] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2023-04-24 19:32:34,132] [INFO] Set up nlp object from config
[2023-04-24 19:32:34,138] [DEBUG] Loading corpus from path: dev.spacy
[2023-04-24 19:32:34,138] [DEBUG] Loading corpus from path: train.spacy
[2023-04-24 19:32:34,139] [INFO] Pipeline: ['textcat']
[2023-04-24 19:32:34,141] [INFO] Created vocabulary
[2023-04-24 19:32:34,141] [INFO] Finished initializing nlp object
[2023-04-24 19:32:36,286] [INFO] Initialized pipeline components: ['textcat']
[2023-04-24 19:32:36,294] [DEBUG] Loading corpus from path: dev.spacy
[2023-04-24 19:32:36,294] [DEBUG] Loading corpus from path: train.spacy


In [16]:
!python -m spacy evaluate ./model/model-best/ ./test.spacy

[i] Using CPU

================================== Results ==================================

TOK                 100.00
TEXTCAT (macro F)   98.85 
SPEED               206796


=========================== Textcat F (per label) ===========================

          P       R       F
POS   98.63   99.12   98.88
NEG   99.08   98.57   98.82


======================== Textcat ROC AUC (per label) ========================

      ROC AUC
POS      1.00
NEG      1.00

